# Rotating Drag Profile
This notebook is intended to connect with the xArm 7 and Bota Force/Torque Sensor. THen it will rotate the arm in set increments and record the drag force for a given duration. The forces and positions will be recorded as csv files in the data folder.

In [1]:
from samplers import DataSource, PosSampler, FTSampler, Experiment
import threading
import time
from xarm.wrapper import XArmAPI

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from motion import continuous_rotation


SDK_VERSION: 1.13.30


Connect to the xArm and Bota force/torque sensor.

In [2]:
arm = XArmAPI('192.168.1.217')
arm.connect()

arm_sampler = PosSampler('arm', arm)
ft_sampler = FTSampler('FT', 'COM6')

ROBOT_IP: 192.168.1.217, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 7,7,XS1304,AC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3


Create and experiment that will take care of sampling from the sensors and saving the data.

In [3]:
experiment = Experiment([arm_sampler, ft_sampler])
experiment.connect_sensors()

Opened serial port COM6
Trying to setup the sensor.
Timestep: 0.0011132812500000001
Frame synced
Collecting...
CRC mismatch received
Frame synced
CRC mismatch received
Frame synced
CRC mismatch received
Frame synced
CRC mismatch received
Frame synced
CRC mismatch received
Frame synced
Collected for 10.000902891159058 seconds.


True

CRC mismatch received
Frame synced
CRC mismatch received
Frame synced
CRC mismatch received
Frame synced


### Run an Experimental Trial
Change the values of the parameters to match the trial being conducted. The arm will move and the files will be named according to the values given.

In [ ]:
ft_sampler.zero()

In [4]:
# These values will control the experiment and be used to name the saveed data.
parameters = {
    'model': 'random-model',
    'angle': None,
    'flume-speed': None,
    'frequency': 0.1,
}

# sleep_duration should be long enough to allow the system to reach steady state before any data is recorded and duration is ho long you would like to record data for. Both times are in seconds.
sleep_duration = 10
duration = (1 / parameters['frequency']) + 10


motion_thread = threading.Thread(target=continuous_rotation, args=(arm, parameters['frequency']))
motion_thread.start()
experiment.new_trial(parameters, duration=duration)
motion_thread.join()

Exception in thread Thread-12 (continuous_rotation):
Traceback (most recent call last):
  File "C:\Users\truma\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\truma\Projects\FHL-Projects\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\truma\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\truma\Projects\FHL-Projects\motion.py", line 93, in continuous_rotation
    while time.perf_counter()-t0 < duration:
                                   ^^^^^^^^
NameError: name 'duration' is not defined


[set_state], xArm is ready to move


In [ ]:
from util import rotate_forces
from scipy.signal import lfilter, lfilter_zi
import numpy as np

df = rotate_forces('data/random-model_0_15.75_FT/FT.csv', 'data/random-model_0_15.75_arm/arm.csv')
window_size = 500
b = (1 / window_size) * np.ones(window_size)
a = 1
x = lfilter(b, a, df['Fx'])

In [ ]:
steps = continuous_rotation(arm, 0.1)